In [1]:
import polars as pl
import pandas as pd
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

c:\Users\Mert\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [20]:
lazy_df_1 = pl.scan_csv(r"C:\Users\Mert\OneDrive\Desktop\Courses\Financial Big Data\data\All_external.csv")

In [5]:
lazy_df_2 = pl.scan_csv(r"C:\Users\Mert\OneDrive\Desktop\Courses\Financial Big Data\data\nasdaq_exteral_data.csv")

In [6]:
lazy_df_2.columns

C:\Users\Mert\AppData\Local\Temp\ipykernel_48184\2180188756.py:1: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  lazy_df_2.columns


['Unnamed: 0',
 'Date',
 'Article_title',
 'Stock_symbol',
 'Url',
 'Publisher',
 'Author',
 'Article',
 'Lsa_summary',
 'Luhn_summary',
 'Textrank_summary',
 'Lexrank_summary']

In [8]:
filtered_df = (
    lazy_df_2
    .select(["Date", "Stock_symbol", "Article_title"])  # Select specific columns
    .filter(pl.col("Article_title").is_not_null())  # Filter null values
    .filter(pl.col("Stock_symbol") == "AAPL")  # Filter Stock_symbol == AAPL 
    .with_columns(pl.col("Date").str.to_datetime("%Y-%m-%d %H:%M:%S UTC"))  # Convert Date to proper datetime format
)

# Collect the LazyFrame into a regular DataFrame
result_df = filtered_df.collect()


shape: (9_338, 3)
┌─────────────────────┬──────────────┬─────────────────────────────────┐
│ Date                ┆ Stock_symbol ┆ Article_title                   │
│ ---                 ┆ ---          ┆ ---                             │
│ datetime[μs]        ┆ str          ┆ str                             │
╞═════════════════════╪══════════════╪═════════════════════════════════╡
│ 2023-12-16 22:00:00 ┆ AAPL         ┆ My 6 Largest Portfolio Holding… │
│ 2023-12-16 22:00:00 ┆ AAPL         ┆ Brokers Suggest Investing in A… │
│ 2023-12-16 21:00:00 ┆ AAPL         ┆ Company News for Dec 19, 2023   │
│ 2023-12-16 21:00:00 ┆ AAPL         ┆ NVIDIA (NVDA) Up 243% YTD: Wil… │
│ 2023-12-16 21:00:00 ┆ AAPL         ┆ Pre-Market Most Active for Dec… │
│ …                   ┆ …            ┆ …                               │
│ 2020-05-28 00:00:00 ┆ AAPL         ┆ The Zacks Analyst Blog Highlig… │
│ 2020-05-28 00:00:00 ┆ AAPL         ┆ Growth Stocks for 2020: Tradin… │
│ 2020-05-28 00:00:00 ┆ AAPL     

In [11]:
a = pd.DataFrame(result_df)
a = a.dropna().drop_duplicates()
a = a.rename({0 : "Date",1 : "Stock_Symbol",2 : "Article_title"}, axis = 1)
a

,Date,Stock_Symbol,Article_title
0,2023-12-16 22:00:00,AAPL,My 6 Largest Portfolio Holdings Heading Into 2...
1,2023-12-16 22:00:00,AAPL,Brokers Suggest Investing in Apple (AAPL): Rea...
2,2023-12-16 21:00:00,AAPL,"Company News for Dec 19, 2023"
3,2023-12-16 21:00:00,AAPL,NVIDIA (NVDA) Up 243% YTD: Will It Carry Momen...
4,2023-12-16 21:00:00,AAPL,"Pre-Market Most Active for Dec 19, 2023 : BMY,..."
...,...,...,...
9333,2020-05-28 00:00:00,AAPL,"The Zacks Analyst Blog Highlights: Apple, Exxo..."
9334,2020-05-28 00:00:00,AAPL,Growth Stocks for 2020: Trading Tech Stocks + ...
9335,2020-05-28 00:00:00,AAPL,Stocks Are Struggling To Post Gains On May 28 ...
9336,2020-05-28 00:00:00,AAPL,Swiss National Bank Ready To Buy Much More Tec...


In [29]:
pd.to_datetime(a[0].max()) - pd.to_datetime(a[0].min())

Timedelta('561 days 22:00:00')